In [1]:
!pip3 install folium
!pip3 install wget


     -------------------------------------- 102.3/102.3 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=e7e9285d3ae4703cd059e5396cc1e2982e5a49770a1948ea867fff9746f38cf2
  Stored in directory: c:\users\acer\appdata\local\pip\cache\wheels\04\5f\3e\46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [76]:
import folium
import wget
import pandas as pd

In [77]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [78]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

In [79]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


In [80]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [81]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )


In [82]:
launch_sitesCoor=launch_sites_df[['Lat','Long']].to_numpy()
Launch_sitesNames=launch_sites_df['Launch Site'].to_numpy()
Launch_sitesNames

array(['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
      dtype=object)

In [83]:
circles=[]
markers1=[]
for i in range(0,4):
	circles.append(folium.Circle(launch_sitesCoor[i,:], radius=1000, color='#0b486b', fill=True).add_child(folium.Popup('NASA Johnson Space Center')))
	markers1.append(folium.map.Marker(
    launch_sitesCoor[i,:],
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#0b486b;"><b>%s</b></div>' % Launch_sitesNames[i],
        )
    ))
	site_map.add_child(circles[i])
	site_map.add_child(markers1[i])
site_map.add_child(circle)
site_map.add_child(marker)

In [84]:
launch_sitesCoor[i,:]

array([  34.63283416, -120.6107455 ])

In [85]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [86]:
marker_cluster = MarkerCluster()

In [87]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [88]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)
colors=spacex_df['marker_color'].to_numpy()
Loc=spacex_df[['Lat','Long']].to_numpy()
# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
	marker2=folium.Marker(
    location=Loc[index,:],
    popup="Timberline Lodge",icon=folium.Icon(color='white', icon_color=colors[index]))
	marker_cluster.add_child(marker2)

site_map

In [89]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [90]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [95]:
distance_coastline = calculate_distance(launch_sitesCoor[1,0],launch_sitesCoor[1,1],28.56261, -80.56789)
distance_city=calculate_distance(launch_sitesCoor[3,0],launch_sitesCoor[3,1],34.0293,-118.22043)
distance_city

229.58354374703305

In [94]:
 distance_marker = folium.Marker(
    loaction=[28.56261, -80.56789],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#000000;"><b>%s</b></div>' % f'{distance_coastline} KM',
        )
    )
distance_marker2 = folium.Marker(
    loaction=[28.55418, -80.58881],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#000000;"><b>%s</b></div>' % f'{distance_coastline} KM',
        )
    )
distance_marker3 = folium.Marker(
    loaction=[28.56276, -80.65392],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#000000;"><b>%s</b></div>' % f'{distance_coastline} KM',
        )
    )
LOCAtion=[(launch_sitesCoor[1,0],launch_sitesCoor[1,1]),(28.56261, -80.56789)]
lines=folium.PolyLine(LOCAtion,weight=1)
site_map.add_child(lines)

LOCAtion2=[(launch_sitesCoor[1,0],launch_sitesCoor[1,1]),(28.55418, -80.58881)]
lines2=folium.PolyLine(LOCAtion2,weight=1)


LOCAtion3=[(launch_sitesCoor[2,0],launch_sitesCoor[2,1]),(28.56276, -80.65392)]
lines3=folium.PolyLine(LOCAtion3,weight=1)


LOCAtion4=[(launch_sitesCoor[2,0],launch_sitesCoor[2,1]),(28.56261, -80.56789)]
lines4=folium.PolyLine(LOCAtion4,weight=1)

LOCAtion5=[(launch_sitesCoor[3,0],launch_sitesCoor[3,1]),(34.63291, -120.62456)]
lines5=folium.PolyLine(LOCAtion5,weight=1)
LOCAtion6=[(launch_sitesCoor[3,0],launch_sitesCoor[3,1]),(34.0293,-118.22043)]
lines6=folium.PolyLine(LOCAtion6,weight=1)
site_map.add_child(lines6)
site_map.add_child(lines5)
site_map.add_child(lines4)
site_map.add_child(lines3)
site_map.add_child(lines2)
site_map.add_child(lines)